In [52]:
# from tpk4170.visualization import Ur5Visualizer
from dataclasses import dataclass
import modern_robotics as mr
import numpy as np
import sympy as sp

np.set_printoptions(suppress=True)

PI_HALF = np.pi / 2
PI = np.pi


In [53]:
@dataclass
class DHLink:
    a: float
    alpha: float
    d: float
    theta: float

    def matrix(self) -> np.array:
        ct = np.cos(self.theta)
        st = np.sin(self.theta)
        ca = np.cos(self.alpha)
        sa = np.sin(self.alpha)
        return np.array(
            [
                [ct, -st * ca, st * sa, self.a * ct],
                [st, ct * ca, -ct * sa, self.a * st],
                [0.0, sa, ca, self.d],
                [0.0, 0.0, 0.0, 1.0],
            ]
        )


In [54]:
class KR6:
    d1, d2, d3, d4 = 400, 455, 420, 80
    a1, a2 = 25, 35
    theta_zero = np.zeros((6, 1))

    M = np.array(
        [[1,  0,  0, a1+d2+d3+d4],
            [0, -1,  0,        0],
            [0,  0, -1,                 a2+d1],
            [0,  0,  0,                              1]])

    Slist = np.array(
        [[0, 0, 1,   0,       0,      0],
            [0, 1, 0, -d1,  0,      a1],
            [0, 1, 0, -d1,  0,     a1+d2],
            [1, 0, 0,   0,   a2+d1,    0],
            [0, 1, 0, -(d1+a2),   0,  a1+d2+d3],
            [1, 0, 0,   0,   a2+d1,    0]]).T

    Blist = (mr.Adjoint(mr.FKinSpace(M, Slist, theta_zero)) @ Slist)

In [57]:
sp.Matrix(KR6().Blist.T)
# KR6().Slist

Matrix([
[  0,    0, -1.0,    0, 980.0,       0],
[  0, -1.0,    0, 35.0,     0, -1005.0],
[  0, -1.0,    0, 35.0,     0, -1460.0],
[1.0,    0,    0,    0,     0,       0],
[  0, -1.0,    0,    0,     0, -1880.0],
[1.0,    0,    0,    0,     0,       0]])

In [35]:
theta = np.array([[0],
                  [0],
                  [PI_HALF],
                  [0],
                  [0],
                  [0]])

alpha = np.array([[0],
                  [-PI_HALF],
                  [0],
                  [-PI_HALF],
                  [PI_HALF],
                  [-PI_HALF]])

d = np.array([[0],
              [400],
              [0],
              [0],
              [420],
              [80]])

a = np.array([[0],
              [25],
              [455],
              [35],
              [0],
              [0]])

dhList = np.concatenate((a, alpha, d, theta), axis=1)
dhList


array([[  0.        ,   0.        ,   0.        ,   0.        ],
       [ 25.        ,  -1.57079633, 400.        ,   0.        ],
       [455.        ,   0.        ,   0.        ,   1.57079633],
       [ 35.        ,  -1.57079633,   0.        ,   0.        ],
       [  0.        ,   1.57079633, 420.        ,   0.        ],
       [  0.        ,  -1.57079633,  80.        ,   0.        ]])

In [36]:
l1 = DHLink(*dhList[0])
l2 = DHLink(*dhList[1])
l3 = DHLink(*dhList[2])
l4 = DHLink(*dhList[3])
l5 = DHLink(*dhList[4])
l6 = DHLink(*dhList[5])


In [37]:
fk_dh_zero = (
    l1.matrix()
    @ l2.matrix()
    @ l3.matrix()
    @ l4.matrix()
    @ l5.matrix()
    @ l6.matrix()
)
fk_dh_zero


array([[   0.,   -0.,   -1., -395.],
       [   0.,   -1.,    0.,   80.],
       [  -1.,   -0.,   -0.,  -90.],
       [   0.,    0.,    0.,    1.]])